## **SISTEMA RECOMENDADOR DE NOTICIAS PARA CLIENTES CORPORATIVOS DE UN BANCO**


**Profesor**: Juan Fernando Perez Bernal

**Integrantes**: Miller Puentes, Joan Esteban Chacón, Grace González, Catalina Cárdenasace González, Catalina Cárdenas


En este documento se presenta el Modelamiento de los datos para el desarrollo de un sistema de recomendación de noticias para los clientes de un banco basado en el sector económico al que pertenece.

### ***Tabla de contenido***

**0.   Instalación y llamada de librerías**

  0.1 Instalación de pyLDAvis

  0.2 Instalación de librerías


**4. Recomendadores**

  4.1 Recomendador: LDA - Stemming

  4.2 Recomendador: LDA - Lematización

**5. Resultados**


## ***0. Instalación y llamada de librerías***

### ***0.1 Instalación de pyLDAvis***

In [1]:
import pandas as pd
import numpy as np
import pyLDAvis

print(pd.__version__)
print(np.__version__)
print(pyLDAvis.__version__)

2.0.0
1.24.3
3.4.0


### ***0.2 Instalación de librerías***

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Importación de librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unidecode
import spacy
from spacy.lang.es.examples import sentences
from unidecode import unidecode
import zipfile
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
import pyLDAvis.gensim_models
from gensim.corpora import Dictionary
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
# from transformers import AutoModel, AutoTokenizer
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from wordcloud import WordCloud
from pprint import pprint
from gensim.models import LdaModel

# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

# Cargar el modelo de spacy para el español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\relat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## ***4. Recomendadores***

Para contruir el recomendador se usa un sistema de prioridad de acuerdo al sector al que pertenece el cliente, pero se hace una distinción dependiendo del sector al que pertenece el cliente como se muestra a continuación:

* ***Sector Industrial:***
       'manufactura de productos de tocador articulos de aseo',
       'combustibles lubricantes', 'comercio de repuestos',
       'edificaciones', 'gas', 'distribuidores productos consumo masivo',
       'automotores', 'grasas aceites', 'obras de infraestructura',
       'comercio de electrodomesticos', 'ferreteria', 'papel carton',
       'extraccion de carbon', 'plastico', 'maquinaria',
       'imprentas edicion', 'otros servicios financieros',
       'transporte terrestre', 'bancos', 'supermercados',
       'quimicos basicos otros quimicos', 'madera muebles',
       'servicios empresas', 'carnes frias',
       'productos metalicos', 'transporte maritimo', 'agroquimicos',
       'comisionistas de bolsa', 'cacao', 'servicios personas',
       'establecimientos educativos', 'comercio de variedades vestuario',
       'calzado marroquineria', 'confecciones',
       'acueducto alcantarillado', 'medios', 'fondos de pensiones',
       'transporte aereo', 'fiduciarias', 'molineria derivados',
       'alimentos concentrados', 'seguros', 'valor agregado',
       'extraccion de petroleo', 'textiles', 'electricidad', 'conservas',
       'ganaderia porcicultura', 'cajas de compensacion',
       'industria basica de hierro acero', 'productos lacteos',
       'pesca atun', 'aseo', 'otras actividades mineras',
       'companias de financiamiento comercial', 'flores',
       'comercio de quimicos agroquimicos', 'banano',
       'productos ceramicos', 'vidrio', 'telefonia fija',
       'bebidas no alcoholicas', 'equipo de transporte',
       'repuestos caucho', 'hoteles', 'cemento', 'bebidas alcoholicas',
       'palma africana', 'otros cultivos', 'otros manufactura',
       'resinas plasticas', 'papeles suaves', 'avicultura', 'arroz',
       'confiteria', 'azucar', 'tabaco', 'comercio de cafe',
       'pinturas barnices'

En el sector industrial la prioridad de las noticias se encuentran por:

1.   `Macroeconómicas`
2.   `Politica`
3.   `Sostenibilidad`
4.   `Innovación`
5.   `Regulaciones`
6.   `Alianzas`
7.   `Reputación`
8.   `Cultura`
9.   `Salud`
10.  `Deporte`

* Sector Salud:
       'ips','comercio de farmaceuticos', 'eps salud prepagada salud',
       'manufactura de productos farmaceuticos'

En el sector de la salud la prioridad y los pesos están función de:

1.   `Salud`
2.   `Sostenibilidad`
3.   `Innovación`
4.   `Regulaciones`
5.   `Deporte`
6.   `Cultura`
7.   `Reputación`
8.   `Macroeconómicas`
9.   `Politica`
10.  `Alianzas`


Nota: La categoría 'Otra' se indica que la eliminemos de las sugerencias ya que no son relevantes para el negocio.

### ***4.1 Recomendador: LDA - Stemming***

In [4]:
# Se carga el set de datos trabajado
df_bd_stemming_f2=pd.read_csv('../Data/DC_stemming_tema_asignado.csv')
# Se cargan los atributos de las noticias
df_noticia=pd.read_csv('../Data/noticias.csv')
# Se carga el set de datos Clientes
df_clientes=pd.read_csv('../Data/clientes.csv')


Se crea la columna con el tema de la noticia y se seleccionan las variables más relevantes para el recomendador:

In [5]:
df_noticia_stemming = df_noticia[['news_id', 'news_url_absolute','news_title', 'news_text_content', 'news_init_date']]
df_noticia_stemming = pd.merge(df_noticia_stemming, df_bd_stemming_f2[['news_id','Tema_texto']], on='news_id')
# Se retiran las noticias asignadas como 'Otra'.
df_noticia_stemming = df_noticia_stemming[df_noticia_stemming['Tema_texto'] != 'Otra']
df_noticia_stemming.head(2)

,news_id,news_url_absolute,news_title,news_text_content,news_init_date,Tema_texto
0,news10006,https://www.bluradio.com/economia/precio-dolar...,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...",2022-07-30,Macroeconómicas
1,news10011,https://www.semana.com/economia/macroeconomia/...,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,2022-07-30,Macroeconómicas


In [6]:
print("Valores únicos por NIT: "+str(df_noticia_stemming['news_id'].nunique()))
print("")
print("Valores nulos: ")
print(df_noticia_stemming.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_noticia_stemming['Tema_texto'].value_counts())
print("")
print("Agrupados por fecha: ")
print(df_noticia_stemming['news_init_date'].value_counts())

Valores únicos por NIT: 20312

Valores nulos: 
news_id              0
news_url_absolute    0
news_title           0
news_text_content    0
news_init_date       0
Tema_texto           0
dtype: int64

Agrupados por subsector: 
Tema_texto
Política           4124
Innovación         3373
Alianzas           3050
Macroeconómicas    2935
Cultura            1429
Regulaciones       1260
Deportes           1180
Reputación         1138
Sostenibilidad     1059
Salud               764
Name: count, dtype: int64

Agrupados por fecha: 
news_init_date
2022-07-30    10271
2022-07-15    10041
Name: count, dtype: int64


Sólo se podrá escoger entre dos dias noticias, porque fueron las fechas de recolección.

In [7]:
# Se escribe el sector al que pertenece el cliente
condicion = lambda x: 'Salud' if x in ['ips','comercio de farmaceuticos', 'eps salud prepagada salud','manufactura de productos farmaceuticos'] else 'Industrial'
df_clientes['sector'] = df_bd_stemming_f2['subsec'].apply(condicion)
df_clientes = df_clientes[['nit', 'nombre', 'sector']]
df_clientes.head(2)

,nit,nombre,sector
0,805027024,SUPERMERCADO LA GRAN COLOMBIA SA,Industrial
1,890100026,CAMAGUEY SA,Industrial


In [8]:
print("Valores únicos por NIT: "+str(df_clientes['nit'].nunique()))
print("")
print("Valores nulos: ")
print(df_clientes.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_clientes['sector'].value_counts())

Valores únicos por NIT: 1507

Valores nulos: 
nit       0
nombre    0
sector    0
dtype: int64

Agrupados por subsector: 
sector
Industrial    1429
Salud           78
Name: count, dtype: int64


Gran parte de la población pertenece a otros sectores diferentes al sector salud.

Ahora, se procede a crear las funciones del recomendador:

1. seleccionar_dos_filas_aleatorias: esta función seleccionará dos filas aleatorias de un grupo de noticias.
2. asignar_valor: Esta función asignará el valor que se le debe dar a la noticia de acuerdo a las definiciones del negocio y del sector del cliente.
3. recomendador: Esta función toma como insumo el nit de un cliente, el número de noticias que se le quiere mostrar y el dataframe que se le va a suministrar. La función filtra por su sector y le asigna la relevancia a la notica de acuerdo a la función `asignar_valor` que está en función del sector al que pertenece, para luego ordenarlas por temas y seleccionar 2 noticias de cada topico para presentarla al cliente, también permite visualizar el número de noticias que se desea ver (pero priorizando las de mayor relevancia para el cliente).

In [9]:
def recomendador(nit, num_noticias, fecha, dff1, dff2):
    """Se ingresan los dos df, clientes y noticias para colocar la importancia de la noticia
    según su sector y seleccionar el número de noticias y la fecha
    dff1 = Dataframe clientes
    dff2 = Dataframe de noticias
    """

    # Se les da el valor dependiendo del cliente
    def asignar_valor(row, sector):
        temas = {
            'Salud': [1, 9],
            'Sostenibilidad': [2, 3],
            'Innovación': [3, 4],
            'Regulaciones': [4, 5],
            'Deportes': [5, 10],
            'Cultura': [6, 8],
            'Reputación': [7, 7],
            'Macroeconómicas': [8, 1],
            'Política': [9, 2],
            'Alianzas': [10, 6]
        }

        tema = row['Tema_texto']

        if sector == 'Salud':
            return temas.get(tema, [11, 0])[0]
        else:
            return temas.get(tema, [11, 0])[1]

    # Se copian los df ingresado
    dff1_c = dff1.copy()
    dff2_c = dff2.copy()


    # Se identifica el sector del Nit
    s = dff1_c[dff1_c['nit'] == nit]['sector'].iloc[0]

    # Se le aplica el valor asignado con la función asignar_valor
    dff2_c = dff2_c[dff2_c['news_init_date'] == fecha]
    dff2_c['valor_asignado'] = dff2_c.apply(lambda row: asignar_valor(row, s), axis=1)

    # Ordena el DataFrame por la columna 'valor_asignado'
    dff2_c = dff2_c.sort_values(by='valor_asignado')


    # Agrupa el DataFrame por 'valor_asignado' y aplica la función para seleccionar aleatoriamente dos noticias
    resultados = dff2_c.groupby('valor_asignado', group_keys=False).apply(lambda group: group.sample(n=min(2, len(group))))

    resultados = resultados[[ 'news_init_date','Tema_texto','news_title', 'news_text_content', 'news_url_absolute']]

    resultados = resultados.rename(columns={'news_init_date': 'Fecha', 'Tema_texto': 'Tema de la Noticia','news_title': 'Titular', 'news_text_content':'Noticia', 'news_url_absolute':'Url'})


    resultados.reset_index(drop=True, inplace=True)

    return resultados.head(num_noticias)

In [10]:
# Ejemplo cliente sector Industrial
p = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
p

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,Gobierno Petro pondra mas impuesto a herencias...,Impuesto a las zonas francas Durante un dialog...,https://www.elcolombiano.com/negocios/subira-e...
1,2022-07-30,Macroeconómicas,"""El proyecto de la tributaria busca recaudo co...",La reforma tributaria que se presento el lunes...,https://www.larepublica.co/especiales/reforma-...
2,2022-07-30,Política,Los millonarios bienes que le quitaron a presu...,Centenares de bienes de diferentes tipos que e...,https://www.eltiempo.com/justicia/delitos/exti...
3,2022-07-30,Política,San Luis / Despotismo Burocratico. El Gobierno...,El pasado miercoles se llamo a Asamblea para c...,https://www.laizquierdadiario.com/El-Gobierno-...
4,2022-07-30,Sostenibilidad,Ciudades de China se enfrentan a una decision ...,Los planes de China para acelerar su expansion...,https://www.larepublica.co/globoeconomia/ciuda...
5,2022-07-30,Sostenibilidad,Petroecuador ejecuta II fase de prueba para de...,"Quito, 6 ago (EFE).- La empresa publica de pet...",https://www.swissinfo.ch/spa/ecuador-combustib...
6,2022-07-30,Innovación,La sostenibilidad hace evolucionar a los perfi...,La importancia del desarrollo del sector de l...,https://www.eleconomista.es/ecoaula/noticias/1...
7,2022-07-30,Innovación,Fuertes tormentas acompanadas de granizo en Ba...,Utilizamos cookies para ofrecerte la mejor ex...,https://www.noticieromadrid.es/general/espana/...
8,2022-07-30,Regulaciones,La negociacion colectiva: mas que una obligaci...,"martes, 9 de agosto de 2022 La negociacion col...",https://www.asuntoslegales.com.co/consultorio/...
9,2022-07-30,Regulaciones,Circular 013 de 2022: Regulacion a precios de ...,"Antes de cerrar su periodo, el Ministerio de S...",https://consultorsalud.com/circular-013-2022-r...


In [11]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Clinica Leon XIII se declara en emergencia fun...,Una nueva voz de auxilio por la saturacion de ...,https://www.elcolombiano.com/antioquia/emergen...
1,2022-07-30,Salud,?Quienes se deben vacunar con la quinta dosis ...,El Ministerio de Salud anuncio que implementar...,https://www.ipsuss.cl/ipsuss/actualidad/quiene...
2,2022-07-30,Sostenibilidad,Alertan de la presencia de una planta invasora...,La Coordinadora Ecologista ha exigido al Gobie...,https://www.lavozdeasturias.es/noticia/asturia...
3,2022-07-30,Sostenibilidad,"Eutrofizacion, acidificacion y otros conceptos...","En las ultimas decadas, hemos sido testigos y ...",https://www.elespanol.com/enclave-ods/historia...
4,2022-07-30,Innovación,Una cultura para el liderazgo,Una sociedad que no tiene su dimension humanis...,https://www.larepublica.co/analisis/jose-leona...
5,2022-07-30,Innovación,En la segunda sesion del Foro para la Descentr...,En la segunda sesion del Foro para la Descentr...,https://www.eha.cl/noticia/local/en-la-segunda...
6,2022-07-30,Regulaciones,"El pais necesita mas libertades, menos regulac...",Los funcionarios proceden sin reconocer que ca...,https://www.infobae.com/opinion/2022/08/14/el-...
7,2022-07-30,Regulaciones,TOMAN PROTESTA DOS NUEVAS FUNCIONARIAS DE ...,"Huixquilucan, Mexico, 09 de agosto de 2022.- E...",https://www.ordenadorpolitico.com/toman-protes...
8,2022-07-30,Deportes,Liga Femenina: el plan que podria asegurar el ...,El futbol femenino en Colombia esta en boca de...,https://www.eltiempo.com/deportes/futbol-colom...
9,2022-07-30,Deportes,Gloria del Real Madrid revela que le falto a J...,"Con el inicio de la liga espanola de futbol, l...",https://www.semana.com/deportes/articulo/glori...


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.

Ahora se adecua el set de datos final, para cargarse en la base de datos de AWS.

In [12]:
df_stemming_BD = df_noticia_stemming[['news_init_date','Tema_texto','news_title','news_url_absolute']]

df_stemming_BD = df_stemming_BD.rename(columns={'news_init_date': 'Fecha',
                                  'Tema_texto': 'Tema_noticia',
                                  'news_title': 'Titulo_noticia',
                                  'news_url_absolute' : 'Url'})

df_stemming_BD.head(3)

,Fecha,Tema_noticia,Titulo_noticia,Url
0,2022-07-30,Macroeconómicas,Precio dolar hoy: la cotizacion de la divisa a...,https://www.bluradio.com/economia/precio-dolar...
1,2022-07-30,Macroeconómicas,Es cierto: El presidente Gustavo Petro quiere ...,https://www.semana.com/economia/macroeconomia/...
2,2022-07-30,Política,Gustavo Petro: ?Que dice el informe final de s...,https://elcomercio.pe/respuestas/que/gustavo-p...


In [13]:
df_clientes_BD = df_clientes.rename(columns={'nit' : 'Nit', 'nombre':'Nombre', 'sector':'Sector'})
df_clientes_BD.head(3)

,Nit,Nombre,Sector
0,805027024,SUPERMERCADO LA GRAN COLOMBIA SA,Industrial
1,890100026,CAMAGUEY SA,Industrial
2,801004045,DON POLLO SAS,Industrial


In [14]:
ruta_csv = '../Data/df_stemming_BD.csv'
df_stemming_BD.to_csv(ruta_csv, index=False)

ruta_csv = '../Data/df_clientes_BD.csv'
df_clientes_BD.to_csv(ruta_csv, index=False)

### ***4.2 Recomendador: LDA - Lematización***

In [15]:
# Se carga el set de datos trabajado
df_bd_lemat_f2=pd.read_csv('../Data/DC_lematizado_tema_asignado.csv')

Se crea la columna con el sector y se seleccionan las variables más relevantes para el recomendador:

In [16]:
df_noticia_lemat = df_noticia[['news_id', 'news_url_absolute','news_title', 'news_text_content', 'news_init_date']]
df_noticia_lemat = pd.merge(df_noticia_lemat, df_bd_lemat_f2[['news_id','Tema_texto']], on='news_id')
# Se retiran las noticias asignadas como 'Otra'.
df_noticia_lemat = df_noticia_lemat[df_noticia_lemat['Tema_texto'] != 'Otra']
df_noticia_lemat.head(2)

,news_id,news_url_absolute,news_title,news_text_content,news_init_date,Tema_texto
0,news10006,https://www.bluradio.com/economia/precio-dolar...,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...",2022-07-30,Macroeconómicas
1,news10011,https://www.semana.com/economia/macroeconomia/...,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,2022-07-30,Macroeconómicas


In [17]:
print("Valores únicos por NIT: "+str(df_noticia_lemat['news_id'].nunique()))
print("")
print("Valores nulos: ")
print(df_noticia_lemat.isnull().sum())
print("")
print("Agrupados por subsector: ")
print(df_noticia_lemat['Tema_texto'].value_counts())
print("")
print("Agrupados por fecha: ")
print(df_noticia_lemat['news_init_date'].value_counts())

Valores únicos por NIT: 19813

Valores nulos: 
news_id              0
news_url_absolute    0
news_title           0
news_text_content    0
news_init_date       0
Tema_texto           0
dtype: int64

Agrupados por subsector: 
Tema_texto
Política           3705
Macroeconómicas    3358
Alianzas           3350
Regulaciones       3232
Cultura            1481
Innovación         1465
Reputación         1317
Salud               717
Deportes            651
Sostenibilidad      537
Name: count, dtype: int64

Agrupados por fecha: 
news_init_date
2022-07-30    10083
2022-07-15     9730
Name: count, dtype: int64


Ahora, se procede a consumir las funciones creadas cuando se creó el recomendador por stemming:

In [18]:
# Ejemplo cliente sector Industrial
r = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
r

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,Flash del mercado | ElEconomista.es,Alemania : \- 08:00h. Balanza comercial de ju...,https://www.eleconomista.es/flash/20220803/
1,2022-07-30,Macroeconómicas,Asi afectaria la tributaria de Petro el comedo...,Una de las premisas bajo las cuales se present...,https://www.portafolio.co/economia/reforma-tri...
2,2022-07-30,Política,Aeronautica Civil de Colombia estudiara integr...,La Aeronautica Civil colombiana reporto que re...,https://dfsud.com/ripe/aeronautica-civil-de-co...
3,2022-07-30,Política,Alberto Fernandez se reunio con Gustavo Petro ...,El presidente Alberto Fernandez invito hoy a s...,https://radio3cadenapatagonia.com.ar/alberto-f...
4,2022-07-30,Sostenibilidad,El proyecto de ley sobre el clima de EE. UU. p...,Con la aprobacion de la legislacion climatica ...,https://www.larepublica.co/globoeconomia/el-pr...
5,2022-07-30,Sostenibilidad,"El potencial de Chile, Argentina, Bolivia, Per...",La produccion mundial de litio se ha duplicado...,https://dfsud.com/ripe/el-potencial-de-chile-a...
6,2022-07-30,Innovación,El Bowlus Volterra Electric es el primer vehic...,"Bowlus , el lider mundial en vehiculos recrea...",https://valenciacars.blogspot.com/2022/08/bowl...
7,2022-07-30,Innovación,Espacio de dialogo abrio ministra del Trabajo ...,"Direccion: Carrera 14 No. 99-33 pisos 6, 7, 10...",https://www.mintrabajo.gov.co/espacio-de-dialo...
8,2022-07-30,Regulaciones,De Mendiguren comparo macroeconomia de Argenti...,"El nuevo secretario de Produccion, Jose Ignaci...",https://www.iprofesional.com/economia/367178-i...
9,2022-07-30,Regulaciones,De norte a sur Conectando Territorios: Fundaci...,"+21,000 proyectos en Latinoamerica. +28,600 e...",https://www.bnamericas.com/es/noticias/de-nort...


In [19]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Realizan formacion docente para coordinadores ...,En el marco de formacion a coordinadores de re...,https://sisanjuan.gob.ar/salud-publica/2022-08...
1,2022-07-30,Salud,Alianza entre Fundacio Puigvert y la empresa I...,La Fundacio Puigvert impulsara su investigacio...,https://www.phmk.es/industria/alianza-entre-fu...
2,2022-07-30,Sostenibilidad,Estudiantes ganan premio por filtro de micropl...,A pesar de que investigadores han encontrado m...,https://www.eltiempo.com/vida/medio-ambiente/p...
3,2022-07-30,Sostenibilidad,"Las pistas de aterrizaje secretas, un arma de ...","Desde 2.500 pies de altura, la pista de aterri...",https://www.clarin.com/mundo/pistas-aterrizaje...
4,2022-07-30,Innovación,Huobi recibe luz verde como proveedor de excha...,El exchange de criptomonedas Huobi puede empez...,https://es.cointelegraph.com/news/huobi-gets-g...
5,2022-07-30,Innovación,Los 30 mejores Pijama Mujer Manga Larga cuidad...,Todos somos conscientes de la dificultad a la ...,https://www.infocastelldefels.cat/los-30-mejor...
6,2022-07-30,Regulaciones,Aqui esta el proximo Suzuki Swift 2024: vealo ...,Hace muy pocos dias vimos por primera vez a la...,https://www.elcarrocolombiano.com/noticias/aqu...
7,2022-07-30,Regulaciones,"Para Ti Fashion Talks Vol. 3, Ricky Sarkany y ...",La moda los unio y la vida los hizo amigos ent...,https://www.parati.com.ar/moda/para-ti-fashion...
8,2022-07-30,Deportes,"Myriam Guerrero, la primera mujer que se atrev...",El culpable de su historia se llama Carlos Alf...,https://www.pulzo.com/deportes/futbol-femenino...
9,2022-07-30,Deportes,"""A morder ese medio campo"": Gallese y la bienv...","Volveran a unir fuerzas en el exterior, a la e...",https://depor.com/futbol-peruano/seleccion-per...


El recomendador asigna de manera adecuada la priorización de noticias de acuerdo al sector.


Ahora se adecua el set de datos final, para cargarse en la base de datos de AWS.

In [20]:
df_lemat_BD = df_noticia_lemat[['news_init_date','Tema_texto','news_title','news_url_absolute']]

df_lemat_BD = df_lemat_BD.rename(columns={'news_init_date': 'Fecha',
                                  'Tema_texto': 'Tema_noticia',
                                  'news_title': 'Titulo_noticia',
                                  'news_url_absolute' : 'Url'})

df_lemat_BD.head(3)

,Fecha,Tema_noticia,Titulo_noticia,Url
0,2022-07-30,Macroeconómicas,Precio dolar hoy: la cotizacion de la divisa a...,https://www.bluradio.com/economia/precio-dolar...
1,2022-07-30,Macroeconómicas,Es cierto: El presidente Gustavo Petro quiere ...,https://www.semana.com/economia/macroeconomia/...
2,2022-07-30,Política,Gustavo Petro: ?Que dice el informe final de s...,https://elcomercio.pe/respuestas/que/gustavo-p...


In [21]:
ruta_csv = '../Data/df_lemat_BD.csv'
df_lemat_BD.to_csv(ruta_csv, index=False)

## ***5. Resultados***

En este apartado se visualizan dos resultados de las sugerencias de las noticias para los clientes, uno del sector salud y otro del sector industrial. La idea es mostrar 10 noticias para cada uno de los modelos y revisar la asertividad de las noticias.

**Sector salud - LDA (Stemming)**

In [22]:
# Ejemplo cliente sector Salud
q = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
q

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Mineros podrian recibir el alta medica el viernes,Los dos mineros rescatados este martes de la m...,https://acento.com.do/actualidad/mineros-podri...
1,2022-07-30,Salud,Tras mas de 60 casos en Chile: ?Se puede conta...,El Ministerio de Salud actualizo su ultimo con...,https://www.biobiochile.cl/noticias/salud-y-bi...
2,2022-07-30,Sostenibilidad,Una empresa zamorana desarrolla un sistema par...,"Grupo Tecopy , con sede social en Zamora , ha ...",https://www.elespanol.com/castilla-y-leon/regi...
3,2022-07-30,Sostenibilidad,Ley de Hidrocarburos y los plazos que limitan ...,La Ley de Hidrocarburos actual impide que las ...,https://www.perspectiva.gt/noticias/ley-de-hid...
4,2022-07-30,Innovación,Superan delitos ciberneticos a prevencion Info...,Las nuevas herramientas tecnologicas han permi...,https://infochannel.info/superan-delitos-ciber...
5,2022-07-30,Innovación,El curso de verano de la UEMC homenajeo a Jesu...,Nuestro sitio web utiliza cookies propias y de...,https://www.tecnovino.com/el-vino-y-el-relato-...
6,2022-07-30,Regulaciones,Piden cuentas a Sugese por ecuatorianos buscad...,(CRHoy.com) La Camara de Intermediarios de Se...,https://www.crhoy.com/sin-categoria/piden-cuen...
7,2022-07-30,Regulaciones,MC califica la propuesta para la Casa del Nino...,"El director del ICA , Manuel Cebrian , <<quier...",https://www.laopiniondemurcia.es/cartagena/202...
8,2022-07-30,Deportes,Alexis Sanchez recibiria millonaria indemnizac...,"Al parecer, la teleserie sobre el futuro de Al...",https://diariousach.cl/alexis-sanchez-recibiri...
9,2022-07-30,Deportes,Colombia derroto a Alemania en su debut del Mu...,Luego de lograr un segundo lugar en la Copa Am...,https://conexioncapital.co/colombia-derrota-a-...


**Sector salud - LDA (Lematización)**

In [23]:
# Ejemplo cliente sector Salud
t = recomendador(nit=890101897, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
t

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Salud,Diagnostican 122 ninos con diabetes al ano,En Republica Dominicana se diagnostican122 nin...,https://listindiario.com/la-republica/2022/08/...
1,2022-07-30,Salud,Gestar Salud de Colombia IPS promueve la lacta...,En el marco de la Semana Mundial de la Lactanc...,https://elmeridiano.co/gestar-salud-de-colombi...
2,2022-07-30,Sostenibilidad,Estiman la huella de carbono de la produccion ...,Segun datos del Inventario Nacional de Gases d...,https://webpicking.com/estiman-la-huella-de-ca...
3,2022-07-30,Sostenibilidad,Pamela Kaufman liderara los negocios internaci...,Agencia: Don buenos aires Marca: Tiendamia dig...,https://www.totalmedios.com/nota/49711/pamela-...
4,2022-07-30,Innovación,La industria de los drones - Agencia TSS,Agencia TSS - La reduccion en los costos de lo...,https://www.unsam.edu.ar/tss/la-industria-de-l...
5,2022-07-30,Innovación,Johnson & Johnson retira sus famosos polvos de...,La farmaceutica Johnson & Johnson (J&J;) ha an...,https://www.lagacetadesalamanca.es/salud/johns...
6,2022-07-30,Regulaciones,Paradojas de la vida,Por: Alberto Linero Tener claro que no podemos...,https://diariodelhuila.com/paradojas-de-la-vida/
7,2022-07-30,Regulaciones,La perdida patrimonial total de Comfamiliar es...,Por primera vez en 56 anos la Caja de Compensa...,https://www.lanacion.com.co/la-perdida-patrimo...
8,2022-07-30,Deportes,"""Ahora cualquiera se pone la camiseta"": Harold...",Deportivo Cali no logra levantar cabeza en la ...,https://www.semana.com/deportes/articulo/ahora...
9,2022-07-30,Deportes,Seleccion Colombia femenina sub-20 busca cupo ...,Luego del resonante triunfo (1-0) sobre la tri...,https://www.elcolombiano.com/deportes/futbol/s...


**Sector Industrial - LDA (Stemming)**

In [24]:
# Ejemplo cliente sector Industrial
p = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_stemming)
p

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,Cuanto vale tu auto usado: los precios de agos...,La Camara del Comercio Automotor (CCA) publico...,https://www.infobae.com/economia/2022/08/08/cu...
1,2022-07-30,Macroeconómicas,Celebra index Zona Costa BC coordinacion con a...,La Asociacion de la Industria Maquiladora y de...,https://mexicoindustry.com/noticia/celebra-ind...
2,2022-07-30,Política,Familias damnificadas seran trasladadas a la P...,La Municipalidad de Asuncion se encuentra gest...,https://independiente.com.py/familias-damnific...
3,2022-07-30,Política,"?Quien es Catalina Velasco Campuzano, la nueva...",El presidente Gustavo Petro designo como minis...,https://www.elespectador.com/economia/quien-es...
4,2022-07-30,Sostenibilidad,Semaforos apagados y duchas mas cortas: las me...,"Espana apagara edificios administrativos, reci...",https://www.niusdiario.es/economia/consumo/202...
5,2022-07-30,Sostenibilidad,"Cartagena, un puerto estrategico para abastece...",El puerto de Cartagena es un punto estrategico...,https://www.eluniversal.com.co/economica/carta...
6,2022-07-30,Innovación,Entregaron material didactico a los equipos de...,Desde la Direccion de Programas a traves del P...,https://sisanjuan.gob.ar/salud-publica/2022-08...
7,2022-07-30,Innovación,Apertura de convocatoria para la aprobacion de...,"La Escuela de Ciencias Basicas, Tecnologia e I...",https://noticias.unad.edu.co/index.php/unad-no...
8,2022-07-30,Regulaciones,Potenciar Trabajo: ?como anotarse para cobrar ...,El Ministerio de Desarrollo Social dispuso una...,https://www.cronista.com/economia-politica/pot...
9,2022-07-30,Regulaciones,Pleno de Torrelavega aprueba inicialmente orde...,Torrelavega ha dado hoy un paso mas en la prox...,http://www.cantabriatv.es/articulo/cultura/ple...


**Sector Industrial - LDA (Lematización)**

In [25]:
# Ejemplo cliente sector Industrial
r = recomendador(nit=860003216, num_noticias=10,fecha='2022-07-30',dff1=df_clientes ,dff2=df_noticia_lemat)
r

,Fecha,Tema de la Noticia,Titular,Noticia,Url
0,2022-07-30,Macroeconómicas,Guia para invertir en acciones en Colombia: co...,A pesar de que en anos recientes en el mundo s...,https://www.valoraanalitik.com/2022/08/08/guia...
1,2022-07-30,Macroeconómicas,Originarios concurren a las agroferia en la co...,Alrededor de 20 mil personas de nueve comunida...,https://www.critica.com.pa/nacional/originario...
2,2022-07-30,Política,"""Se requiere eficacia, no peleas"": Ivan Cepeda...",El senador Ivan Cepeda se refirio este sabado ...,https://www.semana.com/politica/articulo/se-re...
3,2022-07-30,Política,Mexico y Colombia inician una nueva era de coo...,La doctora Beatriz Gutierrez Muller y el canci...,https://hojaderutadigital.mx/mexico-y-colombia...
4,2022-07-30,Sostenibilidad,Alerta Patagonia: el grave impacto ambiental d...,El rio Santa Cruz nace en el deshielo de los g...,https://www.infobae.com/america/medio-ambiente...
5,2022-07-30,Sostenibilidad,Storytelling | Asi se usa el carbon de Norte d...,?Sabia que el carbon se usa hasta en la gastro...,https://www.laopinion.com.co/contenido/empresa...
6,2022-07-30,Innovación,RushBet y LaLiga de Espana extienden su socied...,"RushBet , la marca de apuestas deportivas onli...",https://www.yogonet.com/latinoamerica/noticias...
7,2022-07-30,Innovación,Primera temporada de pesca de anchoveta: mucho...,Registrese y obtenga 3 articulos gratis al mes...,https://www.semanaeconomica.com/sectores-empre...
8,2022-07-30,Regulaciones,Ellos transforman estrategias de negocio en te...,""" Tres Astronautas es una empresa que cuenta c...",https://www.elespectador.com/economia/emprendi...
9,2022-07-30,Regulaciones,Gustavo Bolivar presenta accion de nulidad par...,"Este martes, el senador del Pacto Historico Gu...",https://www.semana.com/nacion/articulo/gustavo...
